# Bank Churn ML

Imports

In [ ]:
import numpy as np
import pandas as pd

### Load Data

In [ ]:
t_df = pd.read_csv(r"C:\bank-churn-data\transactions_tm1_e.csv")
t_array = t_df.to_numpy()
c_df = pd.read_csv(r"C:\bank-churn-data\customers_tm1_e.csv")
c_array = c_df.to_numpy()

In [ ]:
display(t_df)